In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [ ]:
# Download CDC model data to designated local file saving path
domain = 'https://www.cdc.gov'
url = 'https://www.cdc.gov/coronavirus/2019-ncov/covid-data/forecasting-us-previous.html'

response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

all_tags = soup.findAll('a')
for tag in all_tags:
    uri = tag['href']
    if 'model-data' in uri:
        res = requests.get(domain + uri)
        content = res.content
        file_name = uri.split('/')[-1]
        csv_file = open('./' + file_name, 'wb')
        csv_file.write(content)
        csv_file.close()

In [ ]:
#To set jupyter note book display options, with 60 columns
pd.options.display.max_columns=60

In [ ]:
# Create the list of file names: filenames
# Statically
# filenames = ['2020-04-13-model-data.csv','2020-04-20-model-data.csv','2020-04-27-model-data.csv','2020-05-04-model-data.csv','2020-05-11-model-data.csv','2020-05-18-model-data.csv','2020-05-25-model-data.csv','2020-06-01-model-data.csv']

# Dynamically
filenames = [path.split('/')[-1] for path in glob.glob(os.getcwd() + '/*model-data.csv')]
print(filenames)

In [ ]:
# Create the list of three DataFrames: dataframes
dataframes = []
for filename in filenames:
    dataframes.append(pd.read_csv(filename))

#concat the list of dataframe together 
agg_models=pd.concat(dataframes)


#add target weeks number to a new columns
agg_models['target_week']=agg_models.target.str.extract('(\d+)')
agg_models.dropna(subset=['target_week'],inplace=True)

#convert target week as int
agg_models['target_week']=agg_models['target_week'].astype(int)
agg_models.dtypes
agg_models['target_week_end_date']=pd.to_datetime(agg_models['target_week_end_date'])


#filter and keep only less and equal 4 weeks target week
agg_models=agg_models[agg_models['target_week'].le(4)]
agg_models=agg_models[~agg_models['target'].str.contains('inc')]

#check inc death 
inc_death_check=agg_models['target'].isin(['inc'])
inc_death_check.describe()

In [ ]:
#Create a dictionary for abbreviation and full name (states, also convert national to US) in order to match with JHU csse
states_abb = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming',
        'National':'US',
        'United States':'US',
        
}


#Change all the abbreviation to full name 
for state in agg_models['location_name']:
    if state in states_abb.keys():
        state_1=states_abb.get(state)
        agg_models['location_name'].replace(state,state_1,inplace=True)
    else:
        pass


#drop duplicates
agg_models=agg_models.drop_duplicates()
#df.dtypes

In [ ]:
#Read data (death data from jhu csse in the newest time)

url ='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
jhu_csse = pd.read_csv(url, index_col=0)
display(jhu_csse.head)


In [ ]:
#Remove some geo columns and group all the counties into state level

jhu_csse_st = jhu_csse.drop(jhu_csse.columns[[0, 1,2, 3,4,6,7,8,9,10,11]], axis=1) 
jhu_csse_st_grouped=jhu_csse_st.groupby(['Province_State']).sum()
display(jhu_csse_st_grouped)

In [ ]:
#Read US total confirm cases from JHU csse 
url ='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
jhu_csse_ustotal = pd.read_csv(url)
jhu_csse_ustotal = jhu_csse_ustotal.drop(jhu_csse_ustotal.columns[[0, 2, 3,4]], axis=1) 


#identify the rows 
jhu_csse_ustotal=jhu_csse_ustotal.loc[jhu_csse_ustotal['Country/Region']=='US']
jhu_csse_ustotal.rename(columns={'Country/Region':'State'},inplace=True)
jhu_csse_st_grouped.rename(columns={'Province_State':'State'},inplace=True)

#change the index as State
jhu_csse_ustotal=jhu_csse_ustotal.set_index('State')



In [ ]:
#append the US total confirm cases into state confirm cases
ad_agg=jhu_csse_st_grouped.append(jhu_csse_ustotal)
ad_agg.reset_index(level=0,inplace=True)
ad_agg.rename(columns={'index':'State'},inplace=True)

In [ ]:
#convert wide data into long data set and unified the date formate 
ad_agg_1=ad_agg.melt(id_vars='State')
ad_agg_1.columns=['State','Date','Death']
ad_agg_1['Date']=pd.to_datetime(ad_agg_1['Date'])
display(ad_agg_1)

In [ ]:

agg_models
agg_models.loc[agg_models['model']=='NotreDame-FRED']

In [ ]:
#Merge!

master=pd.merge(agg_models,ad_agg_1,how='left',left_on=['location_name','target_week_end_date'],right_on=['State','Date'])
master=master.rename(columns={'target_week_end_date':'predict_date','location_name':'State','point':'predict_death','Date':'Actual_date','Death':'Actual_death'})



In [ ]:
master.to_csv('master_predict_acutal3.csv',index=False)

In [ ]:
##Calculation 
clean_data=master.copy()

In [ ]:
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import matplotlib.pyplot as plt
fig, ax=plt.subplots()
ax.plot(clean_data['predict_date'],clean_data['predict_death'],color='r',marker='o',linestyle='')
ax.plot(clean_data['predict_date'],clean_data['Actual_death'],color='b',marker='o',linestyle='')
plt.show()

In [ ]:
clean_data['dif']=clean_data['predict_death']-clean_data['Actual_death']
clean_data['sq_dif']=clean_data['dif']**2

In [ ]:
# Calculate Accuracy by using log
clean_data['accuracy_pct'] = (1 - np.abs(np.log(clean_data['predict_death'].values/clean_data['Actual_death'].values)))

In [ ]:
data_group_state_count=clean_data.groupby(['model','forecast_date','predict_date']).count()
data_group_count=data_group_state_count['State']
data_group_count.columns=['State','drop']


In [ ]:
state_count=data_group_count['State']
state_count
state_sum=clean_data.groupby(['model','forecast_date','predict_date']).sum()
state_sum_1=state_sum['sq_dif']
MSE=state_sum.merge(state_count.to_frame(), how='left',left_index=True,right_index=True)
MSE['MSE']=MSE['sq_dif']/MSE['State']
MSE.sort_values('MSE')
MSE_before530=MSE.reset_index()
# MSE_before530=MSE_before530.loc[MSE_before530['predict_date']<='2020-05-30']
MSE_final=MSE_before530.drop(MSE_before530.columns[[6,8,9,10]],axis=1)
MSE_final['Accuracy_0']=np.abs(np.log(MSE_final['predict_death']/MSE_final['Actual_death']))
MSE_final['Accuracy_1']=1-MSE_final['Accuracy_0']
MSE_final



In [ ]:
MSE_final.loc[MSE_final['State']<=20]

In [ ]:
ensemble_loc=MSE_final.loc[MSE_final['forecast_date']=='2020-04-20']
ensemble_group_created=ensemble_loc.groupby('predict_date').agg({'MSE':'mean'})
ensemble_group=ensemble_group_created.reset_index()
ensemble_group['model']='Ensemble'
ensemble_group['forecast_date']='2020-04-20'
ensemble_loc_append=ensemble_loc.copy()
ensemble_loc_append=pd.concat([ensemble_loc_append,ensemble_group],sort=False)
ensemble_loc_append=ensemble_loc_append.loc[ensemble_loc_append['model']=='Ensemble']
MSE_final=pd.concat([MSE_final,ensemble_loc_append],sort=False)

In [ ]:
from datetime import datetime

# 1. have every date go through validation
# 2. If date format is correct, do nothing
# 3. If date format is not correct, try to parse it with different format and then convert to the correct format

def normalize_date_format(date_str, debug=False):
    expected_date_format = '%Y-%m-%d'
    date_format_variations = ['%m/%d/%Y']

    try:
        datetime.strptime(date_str, expected_date_format)
        return date_str
    except:
        if debug:
            print("Invalid date format found: %s" % date_str)
        for i, date_format_variation in enumerate(date_format_variations):
            try:
                if debug:
                    print("Parsing incorrect date with date format variation %s: %s" % (i+1, date_format_variation))
                date_obj = datetime.strptime(date_str, date_format_variation)
                updated_date_str = date_obj.strftime(expected_date_format)
                if debug:
                    print("Successfully normalized date str to expected format: %s" % updated_date_str)
                return updated_date_str
            except Exception as e:
                if debug:
                    print("Failed to normalize date with date format variation %s." % i+1)



dates = sorted([normalize_date_format(date) for date in list(set(MSE_final['forecast_date'].to_list()))])

print(dates)

In [ ]:
MSE_final['forecast_date'] = MSE_final['forecast_date'].apply(normalize_date_format)
MSE_final

In [ ]:
###heatmap modeulo 

In [ ]:
###Organized and change model names
MSE_final['model'].describe()
Name_arr=MSE_final['model'].unique()
Name_list=Name_arr.tolist()
Name_list

In [ ]:
###model name dictionary
name_dictionary = {
        'COVIDhub-ensemble': 'Ensemble',
        'ensemble forecast': 'Ensemble',
    
        'GT-DeepCOVID': 'GA_Tech',
        'IHME-CurveFit': 'IHME',
        'Imperial-ensemble1': 'Imperial1',
        'Imperial-ensemble2': 'Imperial2',
        'LANL-GrowthRate': 'LANL',
        'MIT_CovidAnalytics-DELPHI': 'MIT',
        'MOBS_NEU-GLEAM_COVID': 'MOBS',
        'NotreDame-FRED': 'NotreDame',
        'UCLA-SuEIR': 'UCLA',
        'UMass-MechBayes': 'UMass-MB',
        'UT Austin': 'UT',
        'UT-Mobility': 'UT',
        'University of Geneva': 'Geneva',
        'YYG-ParamSearch': 'YYG',
        'Geneva-DeterministicGrowth':'Geneva'
        
        
}

#Change all the names that in the dictionary to common model names
for name in MSE_final['model']:
    if name in name_dictionary.keys():
        name_1=name_dictionary.get(name)
        MSE_final['model'].replace(name,name_1,inplace=True)
    else:
        pass


#check unique
MSE_final['model'].unique()   ##all good :)


In [ ]:
###Aggregate function
MSE_agg_mean=MSE_final.groupby(['model','forecast_date']).agg(
    {'MSE':'mean',
    'Accuracy_1':'mean'}
)



###fix datetime (leave it later)
#MSE_agg_mean['forecast_date']=pd.to_datetime(MSE_agg_mean['forecast_date']).dt.strftime('%Y/%m/%d')

MSE_agg_mean.loc[['IHME']]



In [ ]:
##look up Imperial, select between Imperial 1 and Imperial 2 then change the name to Imperial, fill 2020-5-18
MSE_agg_mean.rename(index={'Imperial2':'Imperial'},inplace=True)

In [ ]:
MSEaggmean_exclude=MSE_agg_mean.drop(['UChicago_100','UChicago_40','UChicago_60','UChicago_80','UChicago',
                                    'CU 20% contact reduction','CU 30% contact reduction', 'CU 40% contact reduction',
       'CU-60contact', 'CU-70contact', 'CU-80contact', 'CU-80contact_1x',
       'CU-80contactw','Imperial1','NotreDame'])
MSEaggmean_exclude


###inspected the date 2020-5-11 format is not uniform, change into %y%d%m
# MSEaggmean_exclude.rename(index={'5/11/2020':'2020-05-11'},inplace=True)
MSEaggmean_exclude.sort_index(level=0)

###convert into wide format
MSE_wide=MSEaggmean_exclude.copy()
MSE_wide=pd.pivot_table(MSE_wide,index=['model'],values='MSE',columns='forecast_date')

# MSE_wide=MSE_wide.sort_values(by=['2020-04-13','2020-04-20','2020-04-27','2020-05-04','2020-05-11','2020-05-18','2020-05-25','2020-06-01','2020-06-15','2020-06-22'],
#                                 na_position='last')
MSE_wide=MSE_wide.sort_values(by=dates,
                                na_position='last')

MSE_wide


In [ ]:
import seaborn as sb
import matplotlib.pyplot as plt

In [ ]:
#release index to columns for heatmap dataframe structure

ax=sb.heatmap(MSE_wide,annot=False,linewidth=.5,cmap='RdYlGn_r')
ax.set(xlabel='Forecast Date',ylabel='Model',title='Mean Square Error')
ax.set_xticklabels(ax.get_xticklabels(),rotation=45)
#ax.title('Mean Sqaure Error')
plt.show()

In [ ]:
MSE_final.to_csv('MSE_final.csv', header=True)

In [ ]:
MSE_wide.to_csv('MSE_wide.csv', header=True)